In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor
from catboost import Pool
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


In [2]:
TRAINED_MODEL_FPATH = 'all_data_model.bin'
SAMPLE_SUBMISSION_FPATH = '../input/competitive-data-science-predict-future-sales/sample_submission.csv'
COMBINED_DATA_FPATH = '../input/textdatanumericdatacombined/DATA_WITH_TXT.h5'

X_0=pd.read_hdf(COMBINED_DATA_FPATH,'X_0')
X_1=pd.read_hdf(COMBINED_DATA_FPATH,'X_1')
X_2=pd.read_hdf(COMBINED_DATA_FPATH,'X_2')
y=pd.read_hdf(COMBINED_DATA_FPATH,'y')

X = pd.concat([X_0,X_1,X_2])
del X_0,X_1,X_2
gc.collect()

58

In [3]:
train_columns = X.columns.tolist()

In [4]:
y = y.loc[X.index]


In [5]:
assert y.index.equals(X.index)

In [6]:
# pool_train = Pool(data=train_X,label=train_y)
# pool_val = Pool(data=val_X,label=val_y)

In [7]:
regressor_kwargs = {
    'depth': 14,
    'iterations':100,
    'random_seed':0,
    'learning_rate':0.04,
    'od_type':'Iter',
    'od_wait':300,
    'task_type':'GPU',
                   }


In [8]:

# model = CatBoostRegressor(**regressor_kwargs)

# model.fit(
#     pool_train,
#     eval_set=pool_val,
#     verbose=100,
#     plot=True
# )

In [9]:
# import matplotlib.pyplot as plt
# importances = model.get_feature_importance(prettified=True)
# x = np.array(list(range(len(importances))))
# y = [imp for (_,imp) in importances]
# my_xticks = [col for (col,_) in importances]
# fig, ax = plt.subplots(figsize=(10,30))    
# width = 0.75 # the width of the bars 
# ax.barh(x, y, width, color="blue")
# ax.set_yticks(x+width/2)
# ax.set_yticklabels(my_xticks, minor=False)
# plt.title('Feature Importances')
# plt.xlabel('importance')

In [10]:
# del pool_train, pool_val, train_X, train_y, val_X, val_y

In [11]:
gc.collect()

14

In [13]:
num_models = 1500
regressor_kwargs['iterations'] = num_models

In [ ]:
pool_train = Pool(data=X,label=y)

In [ ]:
model = CatBoostRegressor(**regressor_kwargs)

model.fit(
    pool_train,    
    verbose=100,
    plot=True
)

In [ ]:
del pool_train, X, y

In [ ]:
model.save_model(TRAINED_MODEL_FPATH)

In [ ]:
X_test=pd.read_hdf(COMBINED_DATA_FPATH,'X_test')
assert all(X_test.columns == X.columns)
sample_submissions_df = pd.read_csv(SAMPLE_SUBMISSION_FPATH, index_col=0)

X_test = X_test.loc[sample_submissions_df.index,:]
test_prediction = model.predict(Pool(data=X_test))
test_prediction[test_prediction > 20] = 20
test_prediction[test_prediction < 0]  = 0
sample_submissions_df['item_cnt_month'] = test_prediction
sample_submissions_df.to_csv('submission.csv')



In [ ]:
sample_submissions_df.head()